# Unit 7: AI vs AI — Train a SoccerTwos Team (MA-POCA + Self-Play)

### The environment 🎮
- **SoccerTwos** — a 2v2 soccer environment made by the Unity MLAgents Team
- Your team of 2 agents must score goals while defending your own net

### The library used 📚
- [MLAgents](https://github.com/Unity-Technologies/ml-agents)

### Before you start ✅
- `Session options > Accelerator > None` (CPU is fine — this env doesn't use GPU)
- **Internet must be enabled** (for cloning the repo and pushing to Hub)
- Add your HuggingFace write token as a Kaggle Secret named `HF_TOKEN`
  - Kaggle sidebar → Add-ons → Secrets → Add New Secret
  - Get your token at: https://huggingface.co/settings/tokens

📄 Course page: https://huggingface.co/learn/deep-rl-course/unit7/hands-on  
🏆 Leaderboard: https://huggingface.co/spaces/huggingface-projects/AIvsAI-SoccerTwos  
🎮 Watch your agent: https://huggingface.co/spaces/unity/ML-Agents-SoccerTwos

## Install ML-Agents and the dependencies 🔽

We install ML-Agents in a Python 3.10 virtual environment to avoid compatibility issues with Kaggle's default Python 3.12.

In [1]:
%%capture
# Cell 1: Install Python 3.10
!add-apt-repository ppa:deadsnakes/ppa -y
!apt-get update -q
!apt-get install -y python3.10 python3.10-venv python3.10-distutils -q

# Cell 2: Create venv and install with full dependency resolution
!python3.10 -m venv /kaggle/working/mlagents-env

!/kaggle/working/mlagents-env/bin/pip install --upgrade pip -q

!/kaggle/working/mlagents-env/bin/pip install \
    "mlagents==1.1.0" \
    "mlagents-envs==1.1.0" \
    "numpy==1.23.5" \
    "onnx==1.15.0" \
    "grpcio==1.48.2" \
    "protobuf==3.19.6" \
    torch --index-url https://download.pytorch.org/whl/cpu

In [2]:
# Cell 3: Verify
!/kaggle/working/mlagents-env/bin/mlagents-learn --help | head -50

usage: mlagents-learn [-h] [--env ENV_PATH] [--resume] [--deterministic]
                      [--force] [--run-id RUN_ID] [--initialize-from RUN_ID]
                      [--seed SEED] [--inference] [--base-port BASE_PORT]
                      [--num-envs NUM_ENVS] [--num-areas NUM_AREAS] [--debug]
                      [--env-args ...]
                      [--max-lifetime-restarts MAX_LIFETIME_RESTARTS]
                      [--restarts-rate-limit-n RESTARTS_RATE_LIMIT_N]
                      [--restarts-rate-limit-period-s RESTARTS_RATE_LIMIT_PERIOD_S]
                      [--torch] [--tensorflow] [--results-dir RESULTS_DIR]
                      [--timeout-wait TIMEOUT_WAIT] [--width WIDTH]
                      [--height HEIGHT] [--quality-level QUALITY_LEVEL]
                      [--time-scale TIME_SCALE]
                      [--target-frame-rate TARGET_FRAME_RATE]
                      [--capture-frame-rate CAPTURE_FRAME_RATE]
                      [--no-graphics] [--no-gr

In [ ]:
# # 1. Install Python 3.10
# !add-apt-repository ppa:deadsnakes/ppa -y -q
# !apt-get update -q
# !apt-get install -y python3.10 python3.10-venv python3.10-distutils -q

# # 2. Create venv
# !python3.10 -m venv /kaggle/working/mlagents-env

# # 3. Install everything in one shot with pinned versions
# !/kaggle/working/mlagents-env/bin/pip install -q \
#     mlagents==1.1.0 \
#     mlagents-envs==1.1.0 \
#     numpy==1.23.5 \
#     onnx==1.15.0 \
#     grpcio==1.48.2 \
#     protobuf==3.19.6 \
#     cattrs==1.1.2 \
#     attrs \
#     six \
#     typing_extensions \
#     cloudpickle \
#     gym \
#     h5py \
#     Pillow \
#     pyyaml \
#     absl-py \
#     tensorboard \
#     werkzeug \
#     markdown \
#     --no-deps

# # 4. Install torch CPU-only (small, no deps)
# # !/kaggle/working/mlagents-env/bin/pip install torch --index-url https://download.pytorch.org/whl/cpu -q --no-deps

# # 5. Verify
# !/kaggle/working/mlagents-env/bin/mlagents-learn --help | head -3

## Download and move the SoccerTwos executable 🔽

We download the Linux SoccerTwos environment binary from the course's Google Drive and place it in `./training-envs-executables/SoccerTwos/`.

The executable contains **8 parallel copies** of the environment for faster training.

In [3]:
!mkdir -p ./training-envs-executables/SoccerTwos

In [4]:
%%capture
# Install other required tools
!pip install huggingface_hub gdown -q

# Install virtual display (required for Unity headless on Linux)
!apt-get install -y xvfb -q

# Install git-lfs (required for pushing to HuggingFace Hub)
!apt-get install -y git-lfs -q
!git lfs install

print('✅ Extra dependencies installed!')

In [5]:
# Download Linux SoccerTwos executable from course Google Drive
!gdown "1KuqBKYiXiIcU4kNMqEzhgypuFP5_45CL" -O ./training-envs-executables/SoccerTwos.zip

Downloading...
From (original): https://drive.google.com/uc?id=1KuqBKYiXiIcU4kNMqEzhgypuFP5_45CL
From (redirected): https://drive.google.com/uc?id=1KuqBKYiXiIcU4kNMqEzhgypuFP5_45CL&confirm=t&uuid=d489bdfc-25df-462b-b168-a6ecb1f0228c
To: /kaggle/working/training-envs-executables/SoccerTwos.zip
100%|██████████████████████████████████████| 37.0M/37.0M [00:00<00:00, 76.9MB/s]


In [9]:
%%capture
!unzip -d ./training-envs-executables/SoccerTwos/ ./training-envs-executables/SoccerTwos.zip

In [10]:
# Make sure the executable is accessible
!chmod -R 755 ./training-envs-executables/SoccerTwos/

In [11]:
# Confirm the executable is present
!ls -la ./training-envs-executables/SoccerTwos/

total 33812
drwxr-xr-x 4 root root     4096 Feb 13 20:05 .
drwxr-xr-x 3 root root     4096 Feb 13 20:26 ..
drwxr-xr-x 3 root root     4096 Jan 31  2023 SoccerTwos_BurstDebugInformation_DoNotShip
drwxr-xr-x 7 root root     4096 Feb 13 20:07 SoccerTwos_Data
-rwxr-xr-x 1 root root    14720 Jan 31  2023 SoccerTwos.x86_64
-rwxr-xr-x 1 root root 34584808 Jan 31  2023 UnityPlayer.so


## Create the SoccerTwos config file

We use the **MA-POCA** (`poca`) trainer with **self-play**.

- MA-POCA uses a centralised critic to train cooperative team behaviour
- Self-play pits the team against past versions of itself

**`max_steps` is set to 3,000,000** (~3–4 hrs on Kaggle) which is enough for certification.  
The course default is 5,000,000 but that risks timing out.

You can modify the hyperparameters here to try to get a better ELO score:
- `batch_size`, `hidden_units`, `learning_rate` affect learning stability
- `save_steps`, `swap_steps` in `self_play` affect how often opponents are updated

In [12]:
config_yaml = """
behaviors:
  SoccerTwos:
    trainer_type: poca
    hyperparameters:
      batch_size: 2048
      buffer_size: 20480
      learning_rate: 0.0003
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: constant
    network_settings:
      normalize: false
      hidden_units: 512
      num_layers: 2
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.99
        strength: 1.0
    keep_checkpoints: 5
    max_steps: 3000000
    time_horizon: 1000
    summary_freq: 10000
    self_play:
      save_steps: 50000
      team_change: 200000
      swap_steps: 2000
      window: 10
      play_against_latest_model_ratio: 0.5
      initial_elo: 1200.0
"""
import os
os.makedirs("/kaggle/working/ml-agents/config/poca", exist_ok=True)
with open("/kaggle/working/ml-agents/config/poca/SoccerTwos.yaml", "w") as f:
    f.write(config_yaml)
print("Config written to /kaggle/working/ml-agents/config/poca/SoccerTwos.yaml")

Config written to /kaggle/working/ml-agents/config/poca/SoccerTwos.yaml


## Train our agents 🚀

To train, we launch `mlagents-learn` with four parameters:

1. `mlagents-learn <config>` — path to the hyperparameter config file
2. `--env` — path to the environment executable
3. `--run-id` — name for this training run
4. `--no-graphics` — don't launch visualisation during training

Use `--resume` to continue a previously interrupted run.  
> It will fail first time when you use `--resume` — try running the block again to bypass the error.

**⚠️ Normal behaviour:**
- ELO score may **drop below 1200 for the first ~2M steps** — this is completely normal!
- Agents spend the early steps exploring randomly before learning to play

Training will take approximately **3–4 hours**. Go grab a ☕ — you deserve it 🤗

In [13]:
!/kaggle/working/mlagents-env/bin/mlagents-learn ./config/poca/SoccerTwos.yaml \
    --env=./training-envs-executables/SoccerTwos/SoccerTwos.x86_64 \
    --run-id="SoccerTwos" \
    --no-graphics \
    --force


            ┐  ╖
        ╓╖╬│╡  ││╬╖╖
    ╓╖╬│││││┘  ╬│││││╬╖
 ╖╬│││││╬╜        ╙╬│││││╖╖                               ╗╗╗
 ╬╬╬╬╖││╦╖        ╖╬││╗╣╣╣╬      ╟╣╣╬    ╟╣╣╣             ╜╜╜  ╟╣╣
 ╬╬╬╬╬╬╬╬╖│╬╖╖╓╬╪│╓╣╣╣╣╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╒╣╣╖╗╣╣╣╗   ╣╣╣ ╣╣╣╣╣╣ ╟╣╣╖   ╣╣╣
 ╬╬╬╬┐  ╙╬╬╬╬│╓╣╣╣╝╜  ╫╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╟╣╣╣╙ ╙╣╣╣  ╣╣╣ ╙╟╣╣╜╙  ╫╣╣  ╟╣╣
 ╬╬╬╬┐     ╙╬╬╣╣      ╫╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╟╣╣╬   ╣╣╣  ╣╣╣  ╟╣╣     ╣╣╣┌╣╣╜
 ╬╬╬╜       ╬╬╣╣      ╙╝╣╣╬      ╙╣╣╣╗╖╓╗╣╣╣╜ ╟╣╣╬   ╣╣╣  ╣╣╣  ╟╣╣╦╓    ╣╣╣╣╣
 ╙   ╓╦╖    ╬╬╣╣   ╓╗╗╖            ╙╝╣╣╣╣╝╜   ╘╝╝╜   ╝╝╝  ╝╝╝   ╙╣╣╣    ╟╣╣╣
   ╩╬╬╬╬╬╬╦╦╬╬╣╣╗╣╣╣╣╣╣╣╝                                             ╫╣╣╣╣
      ╙╬╬╬╬╬╬╬╣╣╣╣╣╣╝╜
          ╙╬╬╬╣╣╣╜
             ╙
        
 Version information:
  ml-agents: 1.1.0,
  ml-agents-envs: 1.1.0,
  Communicator API: 1.5.0,
  PyTorch: 2.10.0+cpu
[INFO] Connected to Unity environment with package version 2.3.0-exp.3 and communication version 1.5.0
[INFO] Connected new brain: SoccerTwos?team=1
[INFO] C

In [20]:
%%capture
!/kaggle/working/mlagents-env/bin/pip install onnxscript "protobuf==3.20.3"

In [21]:
!/kaggle/working/mlagents-env/bin/python -c "import onnxscript; import google.protobuf; print('OK')"

OK


In [22]:
!/kaggle/working/mlagents-env/bin/mlagents-learn ./config/poca/SoccerTwos.yaml \
    --env=./training-envs-executables/SoccerTwos/SoccerTwos.x86_64 \
    --run-id="SoccerTwos" \
    --no-graphics \
    --resume


            ┐  ╖
        ╓╖╬│╡  ││╬╖╖
    ╓╖╬│││││┘  ╬│││││╬╖
 ╖╬│││││╬╜        ╙╬│││││╖╖                               ╗╗╗
 ╬╬╬╬╖││╦╖        ╖╬││╗╣╣╣╬      ╟╣╣╬    ╟╣╣╣             ╜╜╜  ╟╣╣
 ╬╬╬╬╬╬╬╬╖│╬╖╖╓╬╪│╓╣╣╣╣╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╒╣╣╖╗╣╣╣╗   ╣╣╣ ╣╣╣╣╣╣ ╟╣╣╖   ╣╣╣
 ╬╬╬╬┐  ╙╬╬╬╬│╓╣╣╣╝╜  ╫╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╟╣╣╣╙ ╙╣╣╣  ╣╣╣ ╙╟╣╣╜╙  ╫╣╣  ╟╣╣
 ╬╬╬╬┐     ╙╬╬╣╣      ╫╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╟╣╣╬   ╣╣╣  ╣╣╣  ╟╣╣     ╣╣╣┌╣╣╜
 ╬╬╬╜       ╬╬╣╣      ╙╝╣╣╬      ╙╣╣╣╗╖╓╗╣╣╣╜ ╟╣╣╬   ╣╣╣  ╣╣╣  ╟╣╣╦╓    ╣╣╣╣╣
 ╙   ╓╦╖    ╬╬╣╣   ╓╗╗╖            ╙╝╣╣╣╣╝╜   ╘╝╝╜   ╝╝╝  ╝╝╝   ╙╣╣╣    ╟╣╣╣
   ╩╬╬╬╬╬╬╦╦╬╬╣╣╗╣╣╣╣╣╣╣╝                                             ╫╣╣╣╣
      ╙╬╬╬╬╬╬╬╣╣╣╣╣╣╝╜
          ╙╬╬╬╣╣╣╜
             ╙
        
 Version information:
  ml-agents: 1.1.0,
  ml-agents-envs: 1.1.0,
  Communicator API: 1.5.0,
  PyTorch: 2.10.0+cpu
[INFO] Connected to Unity environment with package version 2.3.0-exp.3 and communication version 1.5.0
[INFO] Connected new brain: SoccerTwos?team=1
[INFO] C

## Push the agent to the 🤗 Hub

Now that training is done, we push our model to the Hub to:
1. Complete the certification requirement
2. Enter the AI vs AI challenge leaderboard
3. Watch it play against others in the browser

We use **Kaggle Secrets** to safely access the HuggingFace token (no hardcoded tokens!).

Make sure you added `HF_TOKEN` as a Kaggle Secret before running this section.

In [23]:
# from huggingface_hub import notebook_login
# notebook_login()

In [24]:
!git config --global credential.helper store

In [26]:
import os
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi, login

# Fetch token from Kaggle Secrets (add HF_TOKEN in Kaggle sidebar > Add-ons > Secrets)
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

# Login
login(token=hf_token, add_to_git_credential=True)

# Verify identity and permissions
api = HfApi()
user_info = api.whoami()
print(f"Successfully logged in as: {user_info['name']}")
print(f"Token has Write access: {user_info['auth']['type'] == 'WRITE'}")

Successfully logged in as: Chiz
Token has Write access: False


We define 4 parameters for the push:

1. `--run-id` — the name of the training run id (must match what you used above)
2. `--local-dir` — where the agent was saved: `results/<run_id>`
3. `--repo-id` — your HuggingFace repo: `<your_username>/<repo_name>` (auto-created if it doesn't exist)
4. `--commit-message` — git commit message

> **Edit the `--repo-id` below** to use your own HuggingFace username!

In [29]:
!/kaggle/working/mlagents-env/bin/pip install huggingface_hub

  Using cached huggingface_hub-1.4.1-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 10.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 [huggingface_hub] [huggingface_hub]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlagents 1.1.0 requires onnx==1.15.0, but you have onnx 1.17.0 which is incompatible.


In [30]:
# Replace YOUR_HF_USERNAME with your actual HuggingFace username (e.g. "Chiz")
!/kaggle/working/mlagents-env/bin/mlagents-push-to-hf \
    --run-id="SoccerTwos" \
    --local-dir="./results/SoccerTwos" \
    --repo-id="Chiz/poca-SoccerTwos" \
    --commit-message="Unit 7 SoccerTwos MA-POCA"

[INFO] This function will create a model card and upload your SoccerTwos into HuggingFace Hub. This is a work in progress: If you encounter a bug, please send open an issue
[INFO] Pushing repo SoccerTwos to the Hugging Face Hub
Processing Files (0 / 0)      : |                  |  0.00B /  0.00B            
New Data Upload               : |                  |  0.00B /  0.00B            

  ...occerTwos/SoccerTwos-0.pt:   1%|▏             |  14.0B / 1.29kB            


  ...s/SoccerTwos-1499948.onnx:   1%|▏             |   679B / 61.0kB            



  ...cerTwos-1499948.onnx.data:   1%|▏             | 19.6kB / 1.76MB            




  ...rTwos/SoccerTwos-15406.pt:   1%|▏             |  106kB / 9.48MB            





  ...s/SoccerTwos-1999930.onnx:   1%|▏             |   679B / 61.0kB            






  ...cerTwos-1999930.onnx.data:   1%|▏             | 19.6kB / 1.76MB            







  ...s/SoccerTwos-2499340.onnx:   1%|▏             |   679B / 61.0kB            








  ...ce

If everything worked you should see at the end:
```
Your model is pushed to the hub. You can view your model here: https://huggingface.co/YOUR_USERNAME/poca-SoccerTwos
```

## Verify your model is ready for the AI vs AI Challenge ✅

After pushing, check your model page for:
1. The tag **`ML-Agents-SoccerTwos`** — needed to enter the challenge pool
2. A **`SoccerTwos.onnx`** file in the repo

If the tag is missing, edit the model card on HuggingFace and add it manually.

Your model will be picked up in the next AI vs AI matchmaking cycle (~every 4 hours).

In [31]:
from huggingface_hub import HfApi

# Replace with your username
HF_USERNAME = "Chiz"
REPO_NAME   = "poca-SoccerTwos"
repo_id     = f"{HF_USERNAME}/{REPO_NAME}"

api = HfApi()

try:
    info  = api.model_info(repo_id)
    tags  = info.tags or []
    files = list(api.list_repo_files(repo_id))

    print(f"Model  : {repo_id}")
    print(f"Tags   : {tags}")
    print(f"Files  : {files}")

    if "ML-Agents-SoccerTwos" in tags:
        print("\nTag ML-Agents-SoccerTwos present — you will appear on the leaderboard!")
    else:
        print("\nTag ML-Agents-SoccerTwos MISSING — add it manually on your model page.")

    onnx = [f for f in files if f.endswith(".onnx")]
    if onnx:
        print(f"ONNX file found: {onnx}")
    else:
        print("No .onnx file found — push may not have completed correctly.")

    print(f"\nModel page  : https://huggingface.co/{repo_id}")
    print(f"Leaderboard : https://huggingface.co/spaces/huggingface-projects/AIvsAI-SoccerTwos")
    print(f"Watch match : https://huggingface.co/spaces/unity/ML-Agents-SoccerTwos")

except Exception as e:
    print(f"Error: {e}")
    print("Make sure the push cell ran successfully and your username is correct.")

Model  : Chiz/poca-SoccerTwos
Tags   : ['ml-agents', 'tensorboard', 'onnx', 'SoccerTwos', 'deep-reinforcement-learning', 'reinforcement-learning', 'ML-Agents-SoccerTwos', 'region:us']
Files  : ['.gitattributes', 'README.md', 'SoccerTwos.onnx', 'SoccerTwos/SoccerTwos-0.pt', 'SoccerTwos/SoccerTwos-1499948.onnx', 'SoccerTwos/SoccerTwos-1499948.onnx.data', 'SoccerTwos/SoccerTwos-1499948.pt', 'SoccerTwos/SoccerTwos-15406.pt', 'SoccerTwos/SoccerTwos-1999930.onnx', 'SoccerTwos/SoccerTwos-1999930.onnx.data', 'SoccerTwos/SoccerTwos-1999930.pt', 'SoccerTwos/SoccerTwos-2499340.onnx', 'SoccerTwos/SoccerTwos-2499340.onnx.data', 'SoccerTwos/SoccerTwos-2499340.pt', 'SoccerTwos/SoccerTwos-2999870.onnx', 'SoccerTwos/SoccerTwos-2999870.onnx.data', 'SoccerTwos/SoccerTwos-2999870.pt', 'SoccerTwos/SoccerTwos-3000202.onnx', 'SoccerTwos/SoccerTwos-3000202.onnx.data', 'SoccerTwos/SoccerTwos-3000202.pt', 'SoccerTwos/SoccerTwos-499868.onnx.data', 'SoccerTwos/SoccerTwos-999348.onnx.data', 'SoccerTwos/checkpoint.

## Watch your agent play 🎮

Go to https://huggingface.co/spaces/unity/ML-Agents-SoccerTwos

- Select your model as **team blue** (or purple)
- Pick an opponent — try the [baseline model](https://huggingface.co/unity/MLAgents-SoccerTwos) or whoever is top of the leaderboard

Live matches here aren't counted in your ELO, but it's a great way to see how your agent performs!

Don't forget to share your results in the `#ai-vs-ai-challenge` channel on Discord 🔥

## Congrats on finishing Unit 7! 🎉

## Keep Learning, Stay Awesome 🤗